# Basic Capacioty Expansion Practical

In this practical, we aim to build a simple capacity expansion model for hydrogen supoply chain. The code is developed in python, and the demand, fuel price, network data, and zone characterisitcs are given.

<img src="img/HSC.png" width="300"/>

### 1. Loading and Preprocess the data

Let's start by loading the data we have. Load the data from 'Ex03' Folder. Most of the times we need to preprocess the data. Such that, if we might need a parameter later from the data, we set it up now.

In [ ]:
# Import the packages first
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB

In [3]:
# we save the adress to the data directory in a variable, so we can use it later
data_directory = "/workspaces/Supply_Chain_Analytics_2026/Exercise_Files/Ex03"

Fuels = pd.read_csv(f"{data_directory}/Fuels_data.csv")
Gen_data = pd.read_csv(f"{data_directory}/HSC_Gen_Data.csv")
Load = pd.read_csv(f"{data_directory}/HSC_load.csv")
Network = pd.read_csv(f"{data_directory}/HSC_Pipelines.csv")
Zone_data = pd.read_csv(f"{data_directory}/Zone_Char.csv")


In [4]:
# let's have a look at the data -- Remove the comment tags to show the data
Fuels.head()
#Gen_data.head()
#Load.head()
#Network.head()
#Zone_data.head()

# you can also use .describe() to get a statistical summary of the data
Load.describe()

,Time_Index,Load_HSC_Tonne_z1,Load_HSC_Tonne_z2,Load_HSC_Tonne_z3,Load_HSC_Tonne_z4
count,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000
mean,4380.500000,17.906473,8.635753,6.222432,12.485205
std,2528.938512,8.924145,4.265556,3.142557,4.797066
min,1.000000,4.200000,2.400000,1.400000,5.300000
25%,2190.750000,11.200000,5.300000,3.800000,8.700000
50%,4380.500000,17.000000,8.200000,5.900000,11.900000
75%,6570.250000,23.500000,11.400000,8.200000,15.800000
max,8760.000000,58.000000,26.700000,20.300000,32.100000


We need to define the set of generators, storage units, pipelines, and timesteps, to be able to define variables and constraints by them. 

In [5]:
# Let G represent set of all generators in the model.
# in the HSC_Gen_Data, the column 'H_Gen_type' indicates whether a resource is a generator (>0) or storage unit (0). 
# If it is 1, it is a generator that procudes emission suh SMR, if it is 2, it is a generator that produces zero-emission hydrogen via electrolysis.
dfGen = Gen_data[Gen_data['H_Gen_type']>0].copy()
G = dfGen['r_id'].tolist()
GEN = dfGen.set_index('r_id').to_dict(orient='index')

#print(G)

# Let S represent set of all storage units in the model.
dfStorage = Gen_data[Gen_data['H_Gen_type']==0]
S = dfStorage['r_id'].tolist()
STORAGE = dfStorage.set_index('r_id').to_dict(orient='index')
#print(S['Resource'])

# Let I represent set of all pipelines in the model.
I = Network['HSC_Pipelines'].tolist()
PIPE = Network.set_index('HSC_Pipelines').to_dict(orient='index')
#print(I)

# Let T represent set of all timesteps in the model
Demand = Load.set_index('Time_Index')
#print(Demand)
T = Demand.index.tolist()
T_extended = T + [0]
# Set of all zones is represented by Z
Z = Zone_data['Zones'].tolist()
ZONE = Zone_data.set_index('Zones').to_dict(orient='index')


### 2. Create the Model

In [6]:
hsc = gp.Model("HSC_Capacity_Expansion")

Restricted license - for non-production use only - expires 2027-11-29


### 3. Define the Variables

The variables in the model are either related to Capacity, Operation, or Policy. It is common in programming that the variables are named with "camelCase" for better readability. In this way, we start a variable name with lowercase 'v', and each new word starts with capital letter.

In [7]:
######## .......................#############
# Defining the Capacity decision variables #
######## .......................############

# New and Retired Generation Capacity variables
vNewGenCap = hsc.addVars(G, name="NewGenCap", lb=0, vtype=GRB.INTEGER)
vRetGenCap = hsc.addVars(G, name="RetGenCap", lb=0, vtype=GRB.INTEGER)

# New and Retired Storage Capacity variables
vNewStorCap = hsc.addVars(S, name="NewStorCap", lb=0, vtype=GRB.INTEGER)
vRetStorCap = hsc.addVars(S, name="RetStorCap", lb=0, vtype=GRB.INTEGER)

# New and Retired Pipeline Capacity variables
vNewPipe = hsc.addVars(I, name="NewPipeCap", lb=0, vtype=GRB.INTEGER)
vRetPipe = hsc.addVars(I, name="RetPipeCap", lb=0, vtype=GRB.INTEGER)


In [18]:
######## .......................#############
# Defining the Operation decision variables #
######## .......................############

# Generation from generators for each generator and timestep
vGen = hsc.addVars(G, T, name="Gen", lb=0, vtype=GRB.CONTINUOUS) 

# Storage charge and discharge for each storage unit and timestep
vStorCharge = hsc.addVars(S, T, name="StorCharge", lb=0, vtype=GRB.CONTINUOUS) 
vStorDischarge = hsc.addVars(S, T, name="StorDischarge", lb=0, vtype=GRB.CONTINUOUS)
# State of Charge for each storage unit and timestep
vStorSOC = hsc.addVars(S, T_extended, name="StorSOC", lb=0, vtype=GRB.CONTINUOUS)

# Pipeline flow for each pipeline and timestep
vPipeFlow = hsc.addVars(I, T, name="PipeFlow", vtype=GRB.CONTINUOUS) # Note: can be negative for bi-directional flow

In [9]:
######## .......................#############
# Defining the Policy decision variables   #
######## .......................############

# In this simplified example, the only policy variable is the non-served hydrogen demand. With this variabel, the model can choose to no serve a part of the demand if it is too constly to serve it.
# For example, if the cost of electricity is too high and no other generation options are available, the model can choose to not serve a part of the hydrogen demand and pay a penalty, instead of building expensive new capacity.

vNSD = hsc.addVars(Z, T, name="NonServedDemand", lb=0, vtype=GRB.CONTINUOUS)

# We consider a net-zero HSC in this example, so every unit of CO2 emitted must be have a penalty. So, we do not need an emission variable, we just need to calculate the rmission generated from SMRs.

### 3. Objective

As previously mentioned in the lecture slides, the objective of the model is comprised of cost of investment for new capacity, cost of operation, and cost of penalty terms in the system.

In [10]:
# For every resource, total capacity is equal to existing capacity plus new capacity minus retired capacity.
total_gen_cap = {
    g: GEN[g]['Existing_cap_tonne_p_hr'] + vNewGenCap[g] - vRetGenCap[g]
    for g in G
}

total_sto_cap = {
    s: STORAGE[s]['Existing_cap_tonne'] + vNewStorCap[s] - vRetStorCap[s]
    for s in S
}

total_pipe_num = {
    i: PIPE[i]['Existing_Num_Pipes'] + vNewPipe[i] - vRetPipe[i]
    for i in I
}

# Let's define the investment costs
gen_investment_cost = gp.quicksum(
    vNewGenCap[g] * GEN[g]['Inv_cost_tonne_hr_p_yr']
    for g in G  
)
storage_investment_cost = gp.quicksum(
    vNewStorCap[s] * STORAGE[s]['Inv_cost_tonne_p_yr']
    for s in S)

pipeline_investment_cost = gp.quicksum(
    vNewPipe[i] * PIPE[i]['Investment_cost_per_capacity']
    for i in I)

total_investment_cost = gen_investment_cost + storage_investment_cost + pipeline_investment_cost


In [11]:
# Fixed Operation and Maintenance Costs 
gen_fom_cost = gp.quicksum(
    total_gen_cap[g] * GEN[g]['FOM_Cost_p_tonne_p_hr_yr']
    for g in G  
) 

sto_fom_cost = gp.quicksum(
    total_sto_cap[s] * STORAGE[s]['FOM_Cost_p_tonne_p_yr']
    for s in S  
)

pipe_fom_cost = gp.quicksum(
    total_pipe_num[i] * PIPE[i]['FOM_per_capacity']
    for i in I  
)

# Variabnle Operation and Maintenance Costs -- We only consider fuel cost as variable O&M cost in this example

gen_vom_cost = gp.quicksum(
    vGen[g, t] * Fuels[Fuels['Time_Index'] == t][GEN[g]['Fuel']]
    for g in G for t in T
)

total_operation_cost = gen_fom_cost + sto_fom_cost + gen_vom_cost


/tmp/ipykernel_2817/765298014.py:20: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  vGen[g, t] * Fuels[Fuels['Time_Index'] == t][GEN[g]['Fuel']]


In [12]:
# Total Penalty Costs

NSD_Cost = gp.quicksum(
    vNSD[z, t] * ZONE[z]['HSC_NSD_Cost']
    for z in Z for t in T
)

# Emission Cost - calculated based on generator location and emissions
Emission_Cost = gp.quicksum(
    vGen[g, t] * GEN[g]['Emission_per_tonne_H2'] * ZONE[GEN[g]['Zone']]['Emission_cost']
    for g in G for t in T
)


In [13]:
hsc.setObjective(total_investment_cost + total_operation_cost + NSD_Cost + Emission_Cost, GRB.MINIMIZE)

### 4. Constraints 

Now that we have defined the objective, let's define the constraints of the model. We define the constraints within three categories, i.e., capacity constraints, operational constraints, and policy constraints.

In [14]:
# Let's start by defining the capapcity constraints

# 1. Let's make sure that the capacity retiremment does not exceed exisiting capacity
hsc.addConstrs((vRetGenCap[g] <= GEN[g]['Existing_cap_tonne_p_hr'] for g in G), name="Max_Retirement_Gen")
hsc.addConstrs((vRetStorCap[s] <= STORAGE[s]['Existing_cap_tonne'] for s in S), name="Max_Retirement_Stor")
hsc.addConstrs((vRetPipe[i] <= PIPE[i]['Existing_Num_Pipes'] for i in I), name="Max_Retirement_Pipe")


{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>}

In [15]:
# 2. Now let's define the generation capacity constraints - Generation at each timestep cannon exceed total generation capacity
hsc.addConstrs((vGen[g, t] <= total_gen_cap[g] for g in G for t in T), name="Gen_Capacity_Constr")

# the same applies for storage and pipelins
hsc.addConstrs((vStorCharge[s, t] <= total_sto_cap[s] for s in S for t in T), name="Stor_Charge_Capacity_Constr")
hsc.addConstrs((vStorDischarge[s, t] <= total_sto_cap[s] for s in S for t in T), name="Stor_Discharge_Capacity_Constr")

hsc.addConstrs((vPipeFlow[i, t] <= total_pipe_num[i] * PIPE[i]['Max_pipe_cap_tonne'] for i in I for t in T), name="Pipe_Flow_Capacity_Constr_Pos")
hsc.addConstrs((vPipeFlow[i, t] >= -total_pipe_num[i] * PIPE[i]['Max_pipe_cap_tonne'] for i in I for t in T), name="Pipe_Flow_Capacity_Constr_Neg")

{(1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 9): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 14): <gurobi.Constr *Awaiting Model Update*>,
 (1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (1, 16): <gurobi.Constr *Awaiting Model Update*>,
 (1, 17): <gurobi.Constr *Awaiting Model Update*>,
 (1, 18): <gurobi.Constr *Awaiting Model Update*>,
 (1, 19): <gurobi.Constr *Awaiting Model Update*>,
 (1, 20): <gurobi.Constr *Awaiting Model

In [16]:
# 3. Let's define the balance constraints for generation, storage, and pipelines
# Note that the balance constraints are zone specific

for z in Z:
    for t in T:
        hsc.addConstr(
            gp.quicksum(
                vGen[g, t] 
                for g in G 
                if GEN[g]['Zone'] == z
            )
            +
            gp.quicksum(
                vStorDischarge[s, t] 
                for s in S
                if STORAGE[s]['Zone'] == z
            )
            +
            gp.quicksum(
                vPipeFlow[i, t] 
                for i in I 
                if PIPE[i]['To_Zone'] == z
            )
            +
            vNSD[z, t]
            ==
            Demand.loc[t, f'Load_HSC_Tonne_z{z}']
            +
            gp.quicksum(
                vStorCharge[s, t] 
                for s in S
                if STORAGE[s]['Zone'] == z
            )
            +
            gp.quicksum(
                vPipeFlow[i, t] 
                for i in I
                if PIPE[i]['From_Zone'] == z
            )
        
        )


In [ ]:
# 4. Let's define the constraints of the storage units

# before we define the continuity of storage state of charge, we need to define the initial state of charge at t=0, which we assume to be zero
hsc.addConstrs((vStorSOC[s, 0] == 0 for s in S), name="Stor_SOC_Initial")

# The continuity of storage state of charge
hsc.addConstrs(
    (vStorSOC[s, t] == vStorSOC[s, t-1] + vStorCharge[s, t] - vStorDischarge[s, t]
     for s in S for t in T_extended if t !=0), 
    name="Stor_SOC_Continuity"
)

# We also need to make sure the we do not discharge more than the state of charge in the last timestep
hsc.addConstrs((vStorDischarge[s, t] <= vStorSOC[s, t-1] for s in S for t in T), name="Stor_Discharge_Less_SOC")

# The state of charge cannot exceed the total storage capacity
hsc.addConstrs((vStorSOC[s, t] <= total_sto_cap[s] for s in S for t in T), name="Stor_SOC_Capacity_Constr")


{(3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 5): <gurobi.Constr *Awaiting Model Update*>,
 (3, 6): <gurobi.Constr *Awaiting Model Update*>,
 (3, 7): <gurobi.Constr *Awaiting Model Update*>,
 (3, 8): <gurobi.Constr *Awaiting Model Update*>,
 (3, 9): <gurobi.Constr *Awaiting Model Update*>,
 (3, 10): <gurobi.Constr *Awaiting Model Update*>,
 (3, 11): <gurobi.Constr *Awaiting Model Update*>,
 (3, 12): <gurobi.Constr *Awaiting Model Update*>,
 (3, 13): <gurobi.Constr *Awaiting Model Update*>,
 (3, 14): <gurobi.Constr *Awaiting Model Update*>,
 (3, 15): <gurobi.Constr *Awaiting Model Update*>,
 (3, 16): <gurobi.Constr *Awaiting Model Update*>,
 (3, 17): <gurobi.Constr *Awaiting Model Update*>,
 (3, 18): <gurobi.Constr *Awaiting Model Update*>,
 (3, 19): <gurobi.Constr *Awaiting Model Update*>,
 (3, 20): <gurobi.Constr *Awaiting Model

In [ ]:
# 5. The only required constraint for operation of Pipes is the capacity constraint defined above. Let's work on the ramping limits for generators next.
# Note that the ramp constraints are the easiest sets of constraints to define in capacity expansion models, and many other operation constraints existis that we ignore.

hsc.addConstrs((vGen[g, t] - vGen[g, t-1] <= GEN[g]['Ramp_Up_Percentage'] * total_gen_cap[g] for g in G for t in T if t >1), name="Gen_Ramp_Up_Constr")
hsc.addConstrs((vGen[g, t-1] - vGen[g, t] <= GEN[g]['Ramp_Down_Percentage'] * total_gen_cap[g] for g in G for t in T if t >1), name="Gen_Ramp_Down_Constr")

{(1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 9): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 14): <gurobi.Constr *Awaiting Model Update*>,
 (1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (1, 16): <gurobi.Constr *Awaiting Model Update*>,
 (1, 17): <gurobi.Constr *Awaiting Model Update*>,
 (1, 18): <gurobi.Constr *Awaiting Model Update*>,
 (1, 19): <gurobi.Constr *Awaiting Model Update*>,
 (1, 20): <gurobi.Constr *Awaiting Model Update*>,
 (1, 21): <gurobi.Constr *Awaiting Mode

In [ ]:
# 6. Policy constraints

# the amount of non-served demand cannot exceed a certain share of the total demand in each zone and timestep
hsc.addConstrs((vNSD[z,t] <= Demand.loc[t, f'Load_HSC_Tonne_z{z}'] * ZONE[z]['HSC_NSD_Share'] for z in Z for t in T), name="Max_NSD_Constr")



{(1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 9): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 14): <gurobi.Constr *Awaiting Model Update*>,
 (1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (1, 16): <gurobi.Constr *Awaiting Model Update*>,
 (1, 17): <gurobi.Constr *Awaiting Model Update*>,
 (1, 18): <gurobi.Constr *Awaiting Model Update*>,
 (1, 19): <gurobi.Constr *Awaiting Model Update*>,
 (1, 20): <gurobi.Constr *Awaiting Model

In [26]:
hsc.optimize()

Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (linux64 - "Ubuntu 24.04.2 LTS")

CPU model: Intel(R) Xeon(R) Platinum 8370C CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads



GurobiError: Model too large for size-limited license; visit https://gurobi.com/unrestricted for more information